In [1]:
# Set up a chain rule to calculate the loss function @ saturation w/out propagating derivatives through saturation solver
# This was done in the Language of Molecules paper.

# 1_differentiable_saft works when no solvers are involved, i.e. for properties specified with a given (V, T)
# if, instead, we want to solve for saturation conditions, then I have the old problem of 

In [2]:
import Pkg; Pkg.activate(".")

using Revise
using Clapeyron
includet("./saftvrmienn.jl")
# These are functions we're going to overload for SAFTVRMieNN
import Clapeyron: a_res, saturation_pressure, pressure

using Flux
using Plots
using ForwardDiff, DiffResults

using Zygote, ChainRulesCore
using ImplicitDifferentiation

import TaylorDiff

  Activating project at `~/SAFT_ML`


In [3]:
function make_NN_model(Mw, m, σ, λ_a, λ_r, ϵ)
    model = SAFTVRMieNN(
        params = SAFTVRMieNNParams(
            Mw=[Mw],
            segment=[m],
            sigma=[σ],
            lambda_a=[λ_a],
            lambda_r=[λ_r],
            epsilon=[ϵ],
            epsilon_assoc=Float32[],
            bondvol=Float32[],
        )
    )
    return model
end

# Hack to get around Clapeyron model construction API
function make_model(Mw, m, σ, λ_a, λ_r, ϵ)
    model = SAFTVRMie(["methane"])
    
    model.params.Mw[1] = Mw
    model.params.segment[1] = m
    model.params.sigma[1] = σ
    model.params.lambda_a[1] = λ_a
    model.params.lambda_r[1] = λ_r
    model.params.epsilon[1] = ϵ

    return model
end


make_model (generic function with 1 method)

In [4]:
function forward_sat_p(X)
    T = 100.0
    model = make_model(16.04, X...)

    p, Vₗ, Vᵥ = saturation_pressure(model, T)
    
    return [p, Vₗ, Vᵥ]
end

# Saturation conditions:
#* At conditions specified by y, these two conditions are satisfied
#? But how do I turn this into an optimality condition with the same
#? dimension as X
#! Instead define as chain rule, taking ForwardDiff of these expressions
function conditions_sat_p(X, y)
    T = 100.0
    # Mw, m, σ, λ_a, λ_r, ϵ = X
    model = make_NN_model(16.04, X...)
    
    p†, Vₗ† , Vᵥ† = y
    p† = -(eos(model, Vᵥ†, T) - eos(model, Vₗ†, T))/(Vᵥ† - Vₗ†);

    # Compute ∂p∂V
    _, back = Zygote.pullback(V -> a_res(model, V, T), V)

    
    # ∂p∂Vₗ = -Zygote.gradient(V -> a_res(model, V, T), Vₗ†)[1]
    Vₗ = Vₗ† - (pressure(model, Vₗ†, T) - p†)/∂p∂Vₗ
    # Vl_obj = 
# function Clapeyron.pressure(model::SAFTVRMieNN, V, T, z)
#     f_a(V) = a_res(model, V, T, z)
#     p = -Zygote.gradient(f_a, V)[1]
#     return p
# end
    # return [obj, obj, obj]
    return [obj,]
end

impl_saturation_pressure = ImplicitFunction(forward_sat_p, conditions_sat_p)#, linear_solver=DirectLinearSolver())

ErrorException: syntax: invalid character "†" near column 6

In [5]:
X = [1.0, 3.737e-10, 6.0, 12.504, 152.58]

m = make_model(16.04, X...)
m2 = make_NN_model(16.04, X...)
T = 100.0
@show p, Vₗ, Vᵥ = saturation_pressure(m, T);
@show p_sat = -(eos(m, Vᵥ, T) - eos(m, Vₗ, T))/(Vᵥ - Vₗ)
@show p - p_sat

(p, Vₗ, Vᵥ) = saturation_pressure(m, T) = (34113.760062273235, 3.6950451345479705e-5, 0.02406828436380972)
p_sat = -((eos(m, Vᵥ, T) - eos(m, Vₗ, T))) / (Vᵥ - Vₗ) = 

34113.76006228598
p - p_sat = -1.2747477740049362e-8


-1.2747477740049362e-8

In [5]:
@show p, Vₗ, Vᵥ = saturation_pressure(m, T);

(p, Vₗ, Vᵥ) = saturation_pressure(m, T) = (34113.760062273235, 3.6950451345479705e-5, 0.02406828436380972)


In [6]:
@show y = impl_saturation_pressure(X)
conditions_sat_p(X, y)

UndefVarError: UndefVarError: impl_saturation_pressure not defined

In [7]:
g = Zygote.jacobian(X -> conditions_sat_p(X, y), X)

UndefVarError: UndefVarError: conditions_sat_p not defined

In [8]:
model = SAFTVRMie(["methane"])
saturation_pressure(model, 100.0)

(34113.760062273235, 3.6950451345479705e-5, 0.02406828436380972)

In [10]:
Zygote.jacobian(impl_saturation_pressure, X)
# ForwardDiff.jacobian(impl_saturation_pressure, X)
#! This doesn't seem to have worked... :(

UndefVarError: UndefVarError: impl_saturation_pressure not defined

In [4]:
function impl_sat_p_v2(X)
    T = 100.0
    model = make_model(16.04, X...)
    
    p, Vₗ, Vᵥ = saturation_pressure(model, T)
    
    return [p, Vₗ, Vᵥ]
end

impl_sat_p_v2 (generic function with 1 method)

In [6]:
#! Instead, try implementing as a chain rule?
# function ChainRulesCore.rrule(::typeof(impl_sat_p_v2), X)
function test_rrule(X)
    T = 100.0
    p, Vₗ, Vᵥ = impl_sat_p_v2(X)
    
    function f_pullback(Δy)
        # Functions to calculate p, Vₗ, Vᵥ from a Newton step starting from the converged point
        # This is the approach in Winter et al.
        function f_p(X)
            model = make_NN_model(16.04, X...)
            p2 = -(eos(model, Vᵥ, T) - eos(model, Vₗ, T))/(Vᵥ - Vₗ);
            return p2
        end
        
        function f_V(X, V)
            model = make_NN_model(16.04, X...)
            ∂p∂V = Zygote.gradient(V -> pressure(model, V, T), V)[1]
            V2 = V - (pressure(model, V, T) - p)/∂p∂V
            return V2
        end
        
        ∂p = Zygote.gradient(f_p, X)[1]
        ∂Vₗ = Zygote.gradient(X -> f_V(X, Vₗ), X)[1]
        ∂Vᵥ = Zygote.gradient(X -> f_V(X, Vᵥ), X)[1]
        
        return (NoTangent(), [∂p, ∂Vₗ, ∂Vᵥ])
    end
    
    return [p, Vₗ, Vᵥ], f_pullback
end

# Test the gradient computation
X = [1.0, 3.737e-10, 6.0, 12.504, 152.58]
y, f_pullback = test_rrule(X)

([34113.760062273235, 3.6950451345479705e-5, 0.02406828436380972], f_pullback)

In [30]:
T = 100.0
p, Vₗ, Vᵥ = impl_sat_p_v2(X)

# function f_pullback(Δy)
# Functions to calculate p, Vₗ, Vᵥ from a Newton step starting from the converged point
# This is the approach in Winter et al.
function f_p(X)
    T = 100.0 
    model = make_NN_model(16.04, X...)
    p2 = -(eos(model, Vᵥ, T) - eos(model, Vₗ, T))/(Vᵥ - Vₗ);
    return p2
end

function pressure(model::SAFTVRMieNN, V, T, z=[1.0])
    p = -ForwardDiff.derivative(V -> eos(model, V, T, z), V)
    return p
end

function f_V(X, V)
    T = 100.0 
    model = make_NN_model(16.04, X...)
    # ∂p∂V = Zygote.gradient(V -> pressure(model, V, T), V)[1]
    ∂p∂V = ForwardDiff.derivative(V -> pressure(model, V, T), V)
    V2 = V - (pressure(model, V, T) - p)/∂p∂V
    return V2
end

@show p, f_p(X), ForwardDiff.gradient(f_p, X);
# @show Vₗ, f_V(X, Vₗ), ForwardDiff.gradient(X -> f_V(X, Vₗ), X);
@show Vᵥ, f_V(X, Vᵥ)

# ∂p = Zygote.gradient(f_p, X)[1]
# ∂Vₗ = Zygote.gradient(X -> f_V(X, Vₗ), X)[1]
# ∂Vᵥ = Zygote.gradient(X -> f_V(X, Vᵥ), X)[1]

#     return (NoTangent(), [∂p, ∂Vₗ, ∂Vᵥ])
# end
nothing

(p, f_p(X), ForwardDiff.gradient(f_p, X)) = (34113.760062273235, 34113.7600622861, [-235775.88358427823, -2.738594599593772e14, 61346.898616495506, 12147.672399827, -2130.8750918885917])
(Vᵥ, f_V(X, Vᵥ)) = 

(0.02406828436380972, 0.02406828436380972)


In [26]:
ForwardDiff.derivative(x -> ForwardDiff.derivative(x -> ForwardDiff.derivative(x -> x^10, x), x), 1.0)

720.0

In [11]:
model = make_NN_model(16.04, X...)
f(x) = Zygote.gradient(V -> eos(model, V, T), x)
Zygote.gradient(f, Vₗ)

In [5]:
f_pullback(1.0)

In [10]:
@show impl_sat_p_v2(X);

impl_sat_p_v2(X) = [34113.760062273235, 3.6950451345479705e-5, 0.02406828436380972]


In [11]:
@show Zygote.jacobian(impl_sat_p_v2, X);
# ForwardDiff.jacobian(impl_sat_p_v2, X)